## games info data base columns:

### features to keep:
* first_release_date = first date game was released
* game mode = e.g. single player, multiplayer, etc
* genres
* name
* platforms
* summary = text description of game, to be captured using nlp
* themes
* involved_companies = in development
* keywords = e.g., 'world war 2', 'steampunk'; may be sparse, so unclear if useable
* multiplayer_modes
* franchise (see also franchises)
* game_engines - need to dig further, but likely things like, unity, unreal, etc
* player_perspectives
* storyline

### features to consider:
* age_rating = a code referring to one of the age rating associations, e.g., pegi, esrb
* category = majority over 226k entries have category 0, remained 38k+ has other values; 0 = main_game, 1 = dlc, 2 = expansion...
* external_games = external platforms game is available on, e.g. steam, gog, twitch, epic
* release_dates = provided in-depth information on release dates per region/platform/etc
* language_supports
* status = 0 = released, 2 = alpha, 3 = beta, 4 = early access, etc
* alternative names = might be useful to find near matches
* bundles = which bundles contain this game
* game_localization = might be significant
* collections = the collections this game belongs to (worth exploring)
* collection = the series the game belongs to (worth exploring)
* ports = of the game (worth exploring)
* franchises = other franchises this belongs to (see also franchise)
* forks = IDs of forks of this game (no idea what that means)

### features to drop:
* artworks = can't find it, but has api, so likely can access via api
* cover = cover art
* created_at = when added to igdb
* screenshots = for future development on capturing art style
* slug = unique url-name-string
* tags = auto-generated numbers that allow complex filtering on the igdb api (DROP!)
* updated_at = last time game entry was updated
* url = link to game website
* version_parent = if entry is a version of game, this column has the original parent game id(?)
* version_title = title of this version, e.g., Gold edition
* checksum = hash of the game(?!)
* websites = websites associated with game (url = igdb; websites = developer/publisher)
* follows = no. people following game, depricated
* videos = of gameplay
* hypes = no. people following before release
* dlcs = of the game, their IDs
* expansions
* remakes = IDs of remakes of this game
* expanded_games = of this game, game IDs
* remasters = ID of games that are remasters of this ID
* standalone_expansions = IDs of exactly what it says
* aggregated_rating = based on external critic scores
* aggregated_rating_count = how many external scores
* rating = average rating on igdb
* rating_count = how many ratings on igdb
* total_rating = average rating based on external and igdb rating
* total_rating_count = how many ratings in all (external and igdb)


## Library imports, settings, and global function definitions

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [2]:
# A unique game entry is defined by the unique combination of the following parameters

unique_game = ['name', 'platform', 'release_year']

In [3]:
# confine the data set to certain years, here 2010-2020 (inclusive)

time_boundaries = {'start': 2010, 'end': 2020}

# Function that filters the dataframe for a specified range of years.
# Returns the filtered dataframe.
# Parameters:
# - df = DataFrame, the dataframe to be filtered
# - column = str, name of the column containing the filtered parameter, in this case years
# - time_limits = dict (default = time_boundaries), dict containing the start and end of the range of years to serve as filter
# - include_start = bool (default = True), whether to include the start of the range in the accept range of years
# - include_end = bool (default = False), whether to include the end of the range in the accepted range of years
# - flag = bool (default = False), whether to include rows that have been flagged for missing values

def filter_by_time_boundaries(df, column, time_limits=time_boundaries, include_start=True, include_end=True, flag=False):

    # sets up the filter based on the start of the range
    if include_start:
        start_filter = (df[column] >= time_limits['start'])
    else:
        start_filter = (df[column] > time_limits['start'])
    
    # sets up the filter based on the end of the range
    if include_end:
        end_filter = (df[column] <= time_limits['end'])
    else:
        end_filter = (df[column] < time_limits['end'])
    
    # sets up the filter based on whether to include rows flagge for missing values
    if flag:
        flagged_filter = (df[column] == flag)
        
        # return rows if they fall between start and end (inclusive or exclusive) or if rows were flagged for missing values
        return df[(start_filter & end_filter) | flagged_filter]
    
    # return only rows that fall between start and end (inclusive or exclusive), excluding flagged rows
    return df[start_filter & end_filter]

# Note: currently the project filters out games who did not have a release year (marked with a flag of -1)

In [4]:
# Function converts a variable from one data type to another if possible.
# Returns a value converted into a new datatype, or a flag value (default = -1) if unsuccessful due to ValueError.
# Parameters:
# - value = the value to be cast
# - new_type = the data type of value
# - flag = (default = -1) the value to be returned in case conversion was unsuccessful due to ValueError, e.g. casting np.nan
# as an int

def convert_types(value, new_type, flag=-1):
    try:
        return new_type(value)
    except ValueError:
        return flag

# This function require a lambda function to be passed to the map method, instead of directly mapping this function, i.e.:
# .map(lambda x: convert_types(x, <dtype>, <flag>)) instead of, e.g., .map(convert_types)
    
# Cases this function solves:
# 'user_score' (originally string) had 'tbd' 
# 'release_year' (originally float) had NaN

# NOTE: .astype(dtype, errors='ignore') does almost the same, e.g.:
# sales_df[sales_df['user_score'] == 'tbd']['user_score'].astype(float, errors='ignore')
# In this case, a conversion would take place except where an error occurs, in which case, the original value would be
# returned.

Some of columns of the IGDB data set contain lists of values. These, however, are imported as strings, e.g. '[12623, 6231, 96023]'. Therefore, these strings need to be parsed into proper lists.

In [5]:
# Function parses a string into a list of elements of a specified data type, assuming elements are separated by a comma.
# Returns a list of elements based on the parsed string or the original item to be parsed, if that item is not a string
# (e.g., NaN).
# Parameters:
# - item = str, the string to be parsed
# - dtype = data type (default = int), the data type for each element of the list to be returned
# - ignore_space = bool (default = True), whether to remove spaces from the variable item before parsing it

def pseudo_list_parser(item, dtype=int, ignore_space=True):

    if isinstance(item, str):
        if ignore_space:
            item = item.replace(' ', '')
        return [dtype(x) for x in item.replace('[','').replace(']', '').split(',')]
    
    return item

In [6]:
# Function that maps the pseudo_list_parser function onto a series, thereby parsing each value in the series to a list
# Returns the mapped, parsed column.
# Parameters:
# - column = Series, the column containing the values to be parsed
# - dtype = data type (default = int), the data type for each element of the list to be returned
# - ignore_space = bool (default = True), whether to remove spaces from the variable item before parsing it

def column_parser(column, dtype=int, ignore_space=True):
    
    return column.map(lambda x: pseudo_list_parser(x, dtype, ignore_space))

For the purpose of this project, games are defined as unique based on their name, release year, and platform. Thus, games that share a name but have different release years and/or platforms are considered different games since they can perform differently in terms of sales.

In order to merge the two data sets (one containing the sales data and the other containing the game data), it is necessary to standardize and match the values in platforms between the two data sets. That is also true for the two other values that constitute a unique game, namely, game name and release year. Matching names is done further down this notebook and release years are simply integers and do not require any special treatment.

The following function maps values of a given feature based on one data set onto the values of the same feature in the other data set. It is used to correlate platform values between the two data sets.

In [7]:
# Function maps one set of values to another.
# Returns remapped values. This could be a list of values (if a list was passed to the function) or a single value. If there
# is no corresponding value to use to map, will return NaN.
# Parameters:
# - search_key = the value or list of values to be remapped
# - value_mapping = dict, in which the dict keys are the values passed as search_key and the corresponding dict values
# are the new values.

def map_values(search_key, value_mapping):
    
    # Handling the case in which search_key is a list
    if isinstance(search_key, list):
        
        # initializes the return value as an empty list
        remapped_values = []
        
        # For every item in the search_key list, the loop attempts to find it as a key in the value_mapping dict.
        # If it cannot, it continues to the next iteration (due to the if statement).
        # If it does, it appends the non-None value to remapped_values, which is the return value of the function
        for item in search_key:
            
            # retrieving the new, remapped value or None if it was not found 
            value = value_mapping.get(item, None)
            
            # corresponding values are appended to the return value of the function
            if value:
                remapped_values.append(value)

# FUTURE DEV: switch to the following:
#         for item in search_key:
#             value = value_mapping.get(item, item)
#         remapped_values.append(value)
#         remapped_values = list(set(remapped_values))
#         this would look in value_mapping to see if it finds a match. If it doesn't, it'll "keep" the value as is,
#         and when a merge takes place, rows with these values will be dropped, since there will be no match

        # If no element in the list search_key appears in value_mapping, set the return value to np.nan.
        # If there are elements, make sure to remove any duplicated values.
        # Removing this will make the function return an empty list instead of a NaN, as well as duplicated values otherwise.
#         if not remapped_values:
#             remapped_values = np.nan
#         else:
#             remapped_values = list(set(remapped_values))
    
    # Handling the case in which search_key was not a list (function assumes this means it is a single value)
    else:
        
        # retrieves the new, remapped value or NaN if it was not found
        remapped_values = value_mapping.get(search_key, np.nan)
    
    return remapped_values

# The following code accomplishes this in a similar way but provides less control of how the function behaves.
#     for item in set(l).intersection(value_mapping):
#         remapped_values.append(value_mapping[item])
# Or:
#     remapped_values = [item for item in set(l).intersection(value_mapping)]

Since names are one of the key features that mark a unique game (the others being release year and platform), it is important to standardize the strings that make up the names.

In [8]:
# Function that standardizes strings. It keeps converts diacritics and chinese characters, removes parenthesis, and only
# keeps alphanumeric characters.
# Returns a lowercase, stripped string of alphanumeric characters.
# Parameters:
# - string = str, the string to be standardized

import re
from unidecode import unidecode

def standardize_string(string):

    # if string is not a str, return an empty string
    if not isinstance(string, str):
        return ''
   
    # convert everything to unicode, addressing diacritics as well as chinese characters
    string = unidecode(string)
    
    # remove any non-alphanumeric character or non-space as well as parenthesis (and their enclosed content)
    regex = r'\([^)]*\)|[^a-zA-Z0-9\s]'
    string = re.sub(regex, '', string)
    
    # standardize spacing to retain a single space between words
    string = re.sub(r'\s+', ' ', string)
    
    # change to lowercase and strips whitespaces
    return string.lower().strip()

## Sales data import and preparation

In [9]:
# import the sales data csv into a pandas dataframe

# names of the dataframe columns
sales_columns = ['index', 'name', 'platform', 'release_year', 'genre', 'publisher',
                 'sales_na', 'sales_eu', 'sales_jp', 'sales_other', 'sales_global',
                 'critic_score', 'critic_count', 'user_score', 'user_count', 'developer', 'rating']

# names of the columns to drop
sales_drop_columns = ['critic_score', 'critic_count', 'user_score', 'user_count']

# reading csv into dataframe
df_sales = (pd.read_csv('sales_data.csv', skiprows=1, names=sales_columns, index_col='index')
            .drop(sales_drop_columns, axis=1)
            .drop_duplicates() # There are 209 duplicated rows, which are removed here
            .dropna(subset=['name'])) # There are 2 rows that have NaN as 'name', which makes them useless

In [10]:
# transforming the release year column into an int value
df_sales['release_year'] = df_sales['release_year'].map(lambda x: convert_types(x, int))

# filtering the data by years
df_sales = filter_by_time_boundaries(df_sales, 'release_year')

In [11]:
# remember to comment out the time boundaries filter so I find the -1 year games

In [12]:
#df_sales[df_sales['release_year'] == -1]['platform'].value_counts()

## Section dealing with rows flagged with missing years

Platforms left to do: PS2, Wii, X360, DS, PS3, XB, 2600

In [13]:
#df_sales[(df_sales['release_year'] == -1) & (df_sales['platform'] == '2600')]

In [14]:
#df_sales[df_sales['name'].str.contains('Super Robot Wars OG Saga: Masou Kishin II')]['name'][9739]

In [15]:
# platform = PC

# inversion, 2012
# Homeworld Remastered Collection, 2015
# WRC: FIA World Rally Championship, 2010
# GRID, 2019
# Clockwork Empires, 2016
# Dead Island: Riptide, 2013
# Rocksmith, 2011
# Test Drive Unlimited 2, 2011
# Dead Space 3, 2013
# LEGO Harry Potter: Years 5-7, 2011 | PC, 3DS, PSP
# BioShock 2, 2010
# Tomb Raider, 2013 
# TERA, 2011
# Call of Duty: Black Ops, 2010
    
# Disgaea 3: Absence of Detention, 2011 | PSV
    
# 3DS
# Harvest Moon: The Tale of Two Towns, 2010
# Pet Zombies, 2011
# Face Racers: Photo Finish, 2011
# The Hidden, 2011
# Dream Trigger 3D, 2011
# Beyond the Labyrinth, 2012

# PSP
# Danganronpa: Trigger Happy Havoc, 2010
# Valkyria Chronicles III: Unrecorded Chronicles, 2011
# Super Robot Wars OG Saga: Masou Kishin II - Revelation of Evil God, 2012
# Fullmetal Alchemist: Brotherhood, 2010

## End of section for flagged rows

In [16]:
# string columns to lowercase
regular_string_columns = ['platform', 'genre', 'publisher', 'developer', 'rating']

# standardizes string columns with lowercase
df_sales[regular_string_columns] = df_sales[regular_string_columns].applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [17]:
# standardizes strings in the name column, handling non-alphanumeric characters as well as removing parentheses
df_sales['name'] = df_sales['name'].map(lambda x: standardize_string(x) if isinstance(x, str) else x)

In [18]:
# by grouping rows according to unique_game, I eliminate duplicates while saving the max value for other columns, which can be
# assumed to be more up-to-date (since sales can only increase, not decrease)

df_sales = df_sales.groupby(unique_game).agg('max').reset_index()
# Note that for this dataset, this only affects a single game (name = madden nfl 13, platform = ps3, release_year = 2012),
# whose sales features are not duplicated and therefore not removed above when duplicated rows are removed.

## Mapping genres between games data and sales data

In [19]:
# key = name from df_platforms['id']; value = df_sales['platform']
games_to_sales_platform_dict = {
    'atari 2600': '2600',
    '37': '3ds',
    '137': '3ds', # new 3ds
    '20': 'ds', # nintendo ds
    '159': 'ds', # nintendo dsi
    '9': 'ps3',
    '7': 'ps2',
    '38': 'psp',
    '6': 'pc', # windows
    '13': 'pc', # DOS
    '5': 'wii',
    '12': 'x360',
    '4': 'n64', # nintendo 64
    '21': 'gc', # game cube
    '11': 'xb', # xbox
    '18': 'nes',
    '24': 'gba', # game boy advance
    '46': 'psv', # ps vita ; note also '165' = playstation vr, and '390' = playstation vr2 (both not included in this dict)
    '48': 'ps4',
    '49': 'xone', # xbox one
    '19': 'snes', # super NES
    '59': '2600', # atari 2600
    '41': 'wiiu',
    '32': 'sat', # sega saturn
    '33': 'gb', # game boy
    '22': 'gb', # game boy color
    '136': 'ng', # neo geo ; there are other neo geo variations in df_platforms, but there is no relevant game between 2010-2020    
    '29': 'gen', # sega genesis
    '274': 'pcfx',
    '23': 'dc', # dream cast
    '50': '3do', # 3do interactive multiplayer
    '57': 'ws', # wonderswan
    '86': 'tg16', # turbografx-16/pc engine cd
    '150': 'tg16', # turbografx-16/pc engine
    '78': 'scd', # sega cd
    '35': 'gg' # game gear
}

### Exploration of data

I used to following code to determine the correlations of the platforms between df_games and df_sales.
I do not need to run any of this.

In [20]:
#df_platforms = pd.read_csv('platforms.csv', index_col='Unnamed: 0')

In [21]:
#df_platforms['name'] = df_platforms['name'].str.lower()
#df_platforms['alternative_name'] = df_platforms['alternative_name'].str.lower()

In [22]:
# def filter_by_name(df, name):
#     filt_alt_name = df['alternative_name'].str.contains(name)
#     filt_name = df['name'].str.contains(name)
#     filt_slug = df['slug'].str.contains(name)
    
#     filtered_df = df[filt_alt_name | filt_name | filt_slug]
    
#     return filtered_df

In [23]:
#filter_by_name(df_platforms,'gb')

In [24]:
#df_sales['platform'].unique()

In [25]:
#df_sales[df_sales['platform'] == 'scd']

## games data import and preparation

In [26]:
# columns to drop in the games data set

games_drop_columns = ['artworks',
                      'cover',
                      'created_at',
                      'screenshots',
                      'slug',
                      'tags',
                      'updated_at',
                      'url',
                      'version_parent',
                      'version_title',
                      'checksum',
                      'websites',
                      'follows',
                      'videos',
                      'hypes',
                      'dlcs',
                      'expansions',
                      'remakes',
                      'expanded_games',
                      'remasters',
                      'standalone_expansions',
                      'aggregated_rating',
                      'aggregated_rating_count',
                      'rating',
                      'rating_count',
                      'total_rating',
                      'total_rating_count',
                      'forks',
                      'ports']

In [35]:
# games data set has been split into 10 files due to size, and the following loads all of these ten files as a single dataframe

df_games = pd.DataFrame()

for i in range(0, 10):
    df_partial = pd.read_csv(f'games_data_{i}.csv', low_memory=False, index_col='Unnamed: 0')
    df_games = pd.concat([df_games, df_partial], axis=0)

In [36]:
# dropping columns that I am not using, renaming platforms to platform (for standardizing purposes), and dropping duplicates

df_games = df_games.drop(games_drop_columns, axis=1).rename(columns={'platforms': 'platform'}).drop_duplicates()

In [32]:
# if the games dataset was stored as one file, this command would have done all of the above
# df_games = (pd.read_csv('igdb_raw.csv', low_memory=False, index_col='Unnamed: 0')
#             .drop(games_drop_columns, axis=1)
#             .rename(columns={'platforms': 'platform'})
#             .drop_duplicates())

In [27]:
# convert first_release_date to a datetime data type
df_games['first_release_date'] = pd.to_datetime(df_games['first_release_date'], unit='s')

# creates a new column ('release_year') that contains only the year of release
df_games['release_year'] = df_games['first_release_date'].dt.year.map(lambda x: convert_types(x, int))

# filters data by set time boundaries
df_games = filter_by_time_boundaries(df_games, 'release_year')

In [28]:
# removes all rows that have missing values in any of the fields that define a unique game (name, platform, year)
# this can be done with the unique_game variable only after the creation of the 'release_year' column in the cell above

df_games = df_games.dropna(how='any', subset=unique_game)

In [29]:
# standardizes the name column and drops any row that returned an empty string (i.e. names with only special characters)

df_games['name'] = df_games['name'].map(lambda x: standardize_string(x))
df_games = df_games[df_games['name'] != '']

In [30]:
# Converts the platform column from a single string that looks like a list to an actual list of values.
# Each element of the list are strings themselves, since the dictionary that maps these values has strings as keys

df_games['platform'] = column_parser(df_games['platform'], str)

# changes the values in the platforms column to the values used for platform in the df_sales
df_games['platform'] = df_games['platform'].map(lambda x: map_values(x, games_to_sales_platform_dict))

In [31]:
# Transform rows that have multiple platforms listed into separate rows for each platform, copying all other information

df_games = df_games.explode('platform')

# Any row that has NaN in the platform field is dropped and any row that has empty strings or lists for any of the
# unique_game columns

df_games = df_games.dropna(subset=['platform'])

In [32]:
# converts all columns except for specific ones

columns_not_to_parse = ['id', 'name', 'summary', 'storyline', 'platform', 'release_year']

columns_to_parse = list(df_games.columns)

# removes columns_not_to_parse from columns_to_parse
for column in columns_not_to_parse:
    columns_to_parse.remove(column)

# parse all relevant columns
for column in columns_to_parse:
    df_games[column] = column_parser(df_games[column])

In [33]:
# Multiple Values Aggregator
# An aggregation function for a dataframe groupby method, aggregating by transforming multiple values into a single
# flattened list if possible.
# Return value depends on group argument:
# - if group is empty, returns NaN
# - if group is of length 1, returns the group as is
# - otherwise, returns a flattened list containing all elements in group
# Parameters:
# - group = pandas Series, each column of each group of a groupby operation will be passed to this function as a pandas Series.

def mva(group):
    
    # removes all NaN values
    group = group.dropna()
    
    # if the group is empty, then it means that there were only NaN values in it
    if len(group) == 0:
        return np.nan
    
    # if the group has one element, it is the only one that needs to be returned
    if len(group) == 1:
        return group
    
    # otherwise, there are multiple elements that need to be combined into a list
    aggregated_value = []
    
    for value in group:
        if isinstance(value, list):
            aggregated_value.extend(value)
        else:
            aggregated_value.append(value)
        
    return list(set(aggregated_value))

In [37]:
# sets the behavior of how each column would be aggregated by using dictionaries, where key = column name, and
# value = the function (e.g. mva) or the name of the function (e.g. 'min')

# columns that have multiple values that need to be combined into a flattened list
mva_columns = ['age_ratings', 'category', 'external_games', 'game_modes', 'genres', 'release_dates', 'similar_games',
              'summary', 'themes', 'language_supports', 'involved_companies', 'keywords', 'multiplayer_modes', 'status',
              'alternative_names', 'bundles', 'franchises', 'game_engines', 'player_perspectives', 'game_localizations',
              'collections', 'parent_game', 'collection', 'storyline', 'franchise']
mva_dict = {key: mva for key in mva_columns}

# columns that can be aggregated by taking the minimum value
min_columns = ['id', 'first_release_date']
min_dict = {key: 'min' for key in min_columns}

# creates a single dictionary with the above aggregation behavioral dictionaries.
# This dictionary will be passed onto the agg method of the groupby object
column_aggregation_dict = {**mva_dict, **min_dict}

In [38]:
# aggregates the df_games in order to remove duplicate entries resulting from multiple entries in the dataset itself, likely
# by people opening multiple entries for the same game on igdb.

df_games_agg = df_games.groupby(unique_game).agg(column_aggregation_dict).reset_index()

## merging sales and games data

Since a unique game is defined by the combination of its name, release year, and platform, it is necessary to make sure that all three of these can be matched. Release years and platforms are confined to a certain number of fixed values (e.g., 'pc' or 'ps4' for platforms and 2015 or 2019 for release years). These have already been dealt with above.

Names of games, however, can wildly vary. Even the same game can have different spellings of its name (or could have been input differently, e.g., with colons and hyphens, or using digits vs. roman numerals). Instead of a one-to-one match like with release year and platform, names will be matched by closest match. This introduces a couple of complexities. First, the same name can be used by several games, often referring to older/newer releases of a title. So simply finding the closest match would not work, since this would result in finding the (first) closest match in the data set. This also reveals the second caveat here, which is that multiple names in one data set might match most closely to a single name in the other data set.

These issues can be solved by making the assumption that in a given year and for a given platform, every game will have a different name. Put another way, every game for each year-platform combination will have a distinctively unique name. This allows to look for closest name matches within a given year-platform combination.

In [40]:
# function that returns a dictionary: key = release year, value = list of platforms for which games were made in that key year

# Function that creates a dictionary of all combinations of year-platform in the data set. Keys are the release year and
# values are lists of platforms for which games were made for that key-year (insofar as they appear in the data set).
# Returns this year-platform dictionary.
# Parameters:
# - df: 

from collections import defaultdict

def generate_year_platform_dict(df, year_column='release_year', platform_column='platform'):
    
    return_dict = defaultdict(list)
    
    for year in df[year_column].unique():
        for platform in df[df[year_column] == year][platform_column].unique():
            return_dict[year].append(platform)
    
    return return_dict

In [41]:
# all possible combination of year/platform found in the sales data

sales_comb = generate_year_platform_dict(df_sales)

In [42]:
# all possible combination of year/platform found in the games data

games_comb = generate_year_platform_dict(df_games_agg)

In [43]:
# Creates a dictionary that contains of release year(key) and list of platforms (list) that are shared between the two datasets

shared_comb = {}

for sales_year, sales_platform in sales_comb.items():
    
    # retrieves the list of platforms from df_game according to the year from df_sales; or None if there were no platforms
    # for that sales year
    games_platform = games_comb.get(sales_year, None)
    
    # add to that sales year the platforms that are shared for both datasets for that particular year
    if games_platform:
        shared_comb[sales_year] = list(set(sales_platform).intersection(games_platform))

In [44]:
from rapidfuzz import process

In [45]:
# find the closest string from available choices
# returns closest string found, its score (0 = completely diffent, 100 = identical string), and index of match in choices
# if choices is a series, return the series index

def find_closest_match(string, choices):
    match, score, index = process.extractOne(string, choices)
    return match, score, index

In [46]:
# in order to not match the same string twice, once a match is found, this function removes that match from choices

def find_match_and_remove(string, choices):
    match, score, index = find_closest_match(string, choices)
    choices = choices.drop(index)
    return match, score, index, choices

In [47]:
# this code goes over all year/platform combinations, one at a time, in the sales data and tries to find a match in the game data.
# it slowly builds up a new dataframe that contains those matches, keeping the index in df_games to merge it later on

df_sales_name_matched = pd.DataFrame()

for year, platforms in shared_comb.items():
    for platform in platforms:
#        print(f'Checking {year} + {platform}')
        filtered_sales = df_sales[(df_sales['release_year'] == year) & (df_sales['platform'] == platform)][unique_game]
        filtered_games = df_games_agg[(df_games_agg['release_year'] == year) & (df_games_agg['platform'] == platform)][unique_game]
        
        choices = filtered_games['name']
        filtered_sales['closest_match'], filtered_sales['match_score'], filtered_sales['index_in_df_games_agg'], choices = zip(*filtered_sales['name'].apply(lambda x: find_match_and_remove(x, choices)))
        
        df_sales_name_matched = pd.concat([df_sales_name_matched, filtered_sales], axis=0)

In [48]:
df_sales_name_matched

,name,platform,release_year,closest_match,match_score,index_in_df_games_agg
0,10 minute solution,wii,2010,10 minute solution,100.000000,56
5,101in1 sports party megamix,wii,2010,101in1 sports party megamix,100.000000,182
14,2010 fifa world cup south africa,wii,2010,2010 fifa world cup south africa,100.000000,397
54,active life explorer,wii,2010,active life explorer,100.000000,2059
113,alice in wonderland,wii,2010,alice in wonderland,100.000000,3006
...,...,...,...,...,...,...
5262,zombiu,wiiu,2012,zombiu,100.000000,82949
507,brothers conflict precious baby,psv,2017,baboon,57.000000,6517
3323,phantasy star online 2 episode 4 deluxe package,psv,2017,cosmic star heroine,85.500000,14861
3322,phantasy star online 2 episode 4 deluxe package,ps4,2017,2dark deluxe edition,85.500000,470


In [49]:
# match score of 90 seems to be the threshold where there are many good matches,
# but anything below that results in many misses and only a few good matches.

# then join in the columns from df_sales (after dropping the shared columns, i.e., unique_game) to the matched up rows
df_sales_name_matched = df_sales_name_matched[df_sales_name_matched['match_score'] >= 90].join(df_sales.drop(unique_game, axis=1))

In [50]:
# merges the sales data (also containing the matched indices from the games data) with the aggregated games data

final_columns_to_drop = ['index_in_df_games_agg']

df_final = (pd.merge(df_sales_name_matched,
                     df_games_agg.drop(unique_game, axis=1), left_on='index_in_df_games_agg', right_index=True)
            .drop(final_columns_to_drop, axis=1)
            .reset_index(drop=True))

In [53]:
# write the complete data set into a CSV file.

df_final.to_csv('data_complete.csv', index_label='index')

In [54]:
#df = pd.read_csv('data_complete.csv', index_col='Unnamed: 0')